# Google Earth Engine Panel Data Creation

## Initialize

In [1]:
# !pip install geemap
#!pip install ee

In [2]:
# !pip install uszipcode

In [3]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time


In [4]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [5]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()

## Load Feature Collection - Shapefiles

In [6]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sc_county = counties.filter(ee.Filter.eq('NAME', 'Santa Clara'))

In [7]:
la_county, sc_county

(<ee.featurecollection.FeatureCollection at 0x7f81666723a0>,
 <ee.featurecollection.FeatureCollection at 0x7f81666ddbb0>)

In [8]:
#Income Data
la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")

## Load NAIP Imagery

In [9]:
def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image
     

In [10]:
def get_images(param_dict):
    source_image_collection = param_dict['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

    
    

## Load Labeled Data

In [11]:
## Loading feature collections from Google Earth Engine

water_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0610")
water_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0701_400")
vegetation_trees = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_torrance")
vegetation_grass = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_torrance").limit(400)
turf_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance1")
turf_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance2")
impervious_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance1").limit(35)
impervious_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance2").limit(35)
soil = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

water = water_1.merge(water_2)
turf = turf_1.merge(turf_2)
impervious= impervious_1.merge(impervious_2)


In [12]:
def conditional_water(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(1),feat.set({'landcover': 0}),feat)

def conditional_trees(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(2),feat.set({'landcover': 1}),feat)

def conditional_grass(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(3),feat.set({'landcover': 2}),feat)

def conditional_turf(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(4),feat.set({'landcover': 3}),feat)

def conditional_impervious(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(6),feat.set({'landcover': 4}),feat)

def conditional_soil(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(7),feat.set({'landcover': 5}),feat)

water_tr = water.map(conditional_water)
trees_tr = vegetation_trees.map(conditional_trees)
grass_tr = vegetation_grass.map(conditional_grass)
turf_tr = turf.map(conditional_turf)
impervious_tr = impervious.map(conditional_impervious)
soil_tr = soil.map(conditional_soil)

LABELED_SET = water_tr.merge(trees_tr).merge(grass_tr).merge(turf_tr).merge(impervious_tr).merge(soil_tr)

In [13]:
water_test = ee.FeatureCollection("projects/california-lawn-detection/assets/water_test")
vegetation_trees_test = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_test")
vegetation_grass_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_test")
turf_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_test")
impervious_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_reduced_test")
soil_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

TEST_SET = water_test.merge(vegetation_trees_test).merge(vegetation_grass_test).merge(turf_test).merge(impervious_test).merge(soil_test)

## Build Training Set

In [14]:
training_image_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020],
        'counties': {'la_county': la_county}
         }

TRAINING_IMAGE = get_images(training_image_params)['2020_la_county']

In [15]:
Map.addLayer(TRAINING_IMAGE, {}, 'TRAINING_IMAGE')

In [16]:
# Overlay the points on the imagery to get training.
LABEL = 'landcover'
BANDS = ['R', 
         'G', 
         'B', 
         'N', 
         'NDVI',
         'N_Entropy', 
         'N_Contrast', 
         'N_Gearys']

train_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': LABELED_SET,
  'properties': [LABEL],
  'scale': 1
})

test_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': TEST_SET,
  'properties': [LABEL],
  'scale': 1
})

In [17]:
# print("Training Set Size in Pixels", train_data.aggregate_count('R').getInfo())

In [18]:
# print("Test Set Size in Pixels", test_data.aggregate_count('R').getInfo())

## Machine Learning Model

In [19]:
clf = ee.Classifier.smileRandomForest(numberOfTrees = 230, minLeafPopulation = 50, bagFraction= 0.6)\
                   .train(train_data, LABEL, BANDS)
clf

In [20]:
training_image_classified = TRAINING_IMAGE.select(BANDS)\
                                          .classify(clf)


In [21]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

Map.addLayer(training_image_classified, {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification')

In [22]:
training_image_classified.bandNames().getInfo()

['classification']

In [23]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Binary Classification and Area Calculation

In [24]:
def area_calculation(image, class_number, shape, pixel_scale = 5):

    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    area_sq_km = area_sq_m / 1e6

    return area_sq_km

In [25]:
def area_calculation_city(image, class_number, shape, pixel_scale = 20):

    if type(shape) == str:
        shape = la_cities_land.filter(ee.Filter.eq('NAME', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    area_sq_km = area_sq_m / 1e6

    return area_sq_km

In [26]:
def ndvi_calculation(image, class_number, shape, ref_image, pixel_scale = 5):
    
    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))
        
    ndvi = ref_image.normalizedDifference(['N', 'R'])
    image_clipped = image.clip(shape)
    
    NDVI_for_class = ndvi.updateMask(image_clipped.select('classification').eq(class_number))
    
    reducer = ee.Reducer.mean()\
                        .combine(ee.Reducer.max(),sharedInputs=True)\
                        .combine(ee.Reducer.min(),sharedInputs=True)
    
    
    qty = NDVI_for_class.reduceRegion(
        reducer = reducer, 
        geometry = shape, 
        scale = pixel_scale, 
        maxPixels = 1e13)
    return qty



### Create Panel Data

In [27]:
#import parcel shapes so we can clip by residential areas

la_parcel_shape_filtered = ee.FeatureCollection("projects/california-lawn-detection/assets/LA_County_Parcels_Shape")\
                             .filter(ee.Filter.eq('UseType', 'Residential'))
    
la_parcel_res = la_parcel_shape_filtered.select(ee.List(['AIN', 'SitusCity','SitusZIP','SitusFullA']), 
                                                ee.List(['AIN', 'City','ZipCode','FullAddress']))


In [28]:
#import zipcode shapes so we can clip by zipcodes

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [29]:
def run_inference_city(inference_params):
    
    #unpack inference parameter dictionary
    inference_images = get_images(inference_params)
    residential = inference_params['residential']
    city_list = inference_params['city']
    ndvi = inference_params['ndvi']
    city_shape = inference_params['city_shape']
    residential_shape = inference_params['residential_shape']
    
    #add empty lists to data dictionary
    dictionary = {}
    
    base_keys = ['year','polygon','water_area','vegetation_trees_area', 
        'vegetation_grass_area', 'turf_area', 'impervious_area',
        'soil_area', 'total_area']
    
    ndvi_keys = ['tree_ndvi_mean', 'tree_ndvi_max','tree_ndvi_min',
       'grass_ndvi_mean', 'grass_ndvi_max','grass_ndvi_min']
    
    for i in base_keys:
        dictionary[i] = []
    if ndvi:
        for i in ndvi_keys:
            dictionary[i] = []
    
    
    #warning message about selected options
    if inference_params['residential']:
        print('CLIPPING AREA TO INCLUDE RESIDENTIAL AREAS ONLY')
    if inference_params['ndvi']:
        print('RUNNING INFERENCE INCLUDING NDVI CALCULATIONS')
    if inference_params['residential'] or inference_params['ndvi']:
        print('---------------------------------------------------------------------')
    

    #iterate through data, append to data dictionary 
    for i in city_list:
        for j in list(inference_images.items()):
            image_name = j[0]
            im = j[1]
            if residential:
                im = im.clip(residential_shape)
            imagery = im.select(BANDS).classify(clf)
            name = j[0]

            start = time()
            polygon = la_cities_land.filter(ee.Filter.eq('NAME', i))

            dictionary['year'].append(image_name[:4]) 
            dictionary['polygon'].append(i)

            water_area = area_calculation_city(imagery, 0, polygon, 20)
            dictionary['water_area'].append(water_area)

            vegetation_trees_area = area_calculation_city(imagery, 1, polygon, 20)
            dictionary['vegetation_trees_area'].append(vegetation_trees_area)

            vegetation_grass_area = area_calculation_city(imagery, 2, polygon, 20)
            dictionary['vegetation_grass_area'].append(vegetation_grass_area)

            turf_area = area_calculation_city(imagery, 3, polygon, 20)
            dictionary['turf_area'].append(turf_area)

            impervious_area = area_calculation_city(imagery, 4, polygon, 20)
            dictionary['impervious_area'].append(impervious_area)

            soil_area = area_calculation_city(imagery, 5, polygon, 20)
            dictionary['soil_area'].append(soil_area)

            total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
            dictionary['total_area'].append(total_area)

            end = time()
            print(f'City: {i}, Year: {j[0][:4]} ::: completed in {end-start} seconds.')
            
    return dictionary
              
              

In [30]:
def run_inference(inference_params):
    
    #unpack inference parameter dictionary
    inference_images = get_images(inference_params)
    residential = inference_params['residential']
    zipcode_list = inference_params['zipcodes']
    ndvi = inference_params['ndvi']
    zipcode_shape = inference_params['zipcode_shape']
    residential_shape = inference_params['residential_shape']
    
    #add empty lists to data dictionary
    dictionary = {}
    
        
    base_keys = ['year','polygon','water_area','vegetation_trees_area', 
        'vegetation_grass_area', 'turf_area', 'impervious_area',
        'soil_area', 'total_area']
    
    ndvi_keys = ['tree_ndvi_mean', 'tree_ndvi_max','tree_ndvi_min',
       'grass_ndvi_mean', 'grass_ndvi_max','grass_ndvi_min']
    
    for i in base_keys:
        dictionary[i] = []
    if ndvi:
        for i in ndvi_keys:
            dictionary[i] = []
    
    #warning message about selected options
    if inference_params['residential']:
        print('CLIPPING AREA TO INCLUDE RESIDENTIAL AREAS ONLY')
    if inference_params['ndvi']:
        print('RUNNING INFERENCE INCLUDING NDVI CALCULATIONS')
    if inference_params['residential'] or inference_params['ndvi']:
        print('---------------------------------------------------------------------')
    

    #iterate through data, append to data dictionary 
    for i in zipcode_list:
        for j in list(inference_images.items()):
            image_name = j[0]
            im = j[1]
            if residential:
                im = im.clip(residential_shape)
            imagery = im.select(BANDS).classify(clf)
            name = j[0]

            start = time()
            polygon = zipcode_shape.filter(ee.Filter.eq('ZipCode', i))

            dictionary['year'].append(image_name[:4]) 
            dictionary['polygon'].append(i)

            water_area = area_calculation(imagery, 0, polygon, 5)
            dictionary['water_area'].append(water_area)

            vegetation_trees_area = area_calculation(imagery, 1, polygon, 5
            dictionary['vegetation_trees_area'].append(vegetation_trees_area)

            vegetation_grass_area = area_calculation(imagery, 2, polygon, 5)
            dictionary['vegetation_grass_area'].append(vegetation_grass_area)

            turf_area = area_calculation(imagery, 3, polygon, 5)
            dictionary['turf_area'].append(turf_area)

            impervious_area = area_calculation(imagery, 4, polygon, 5)
            dictionary['impervious_area'].append(impervious_area)

            soil_area = area_calculation(imagery, 5, polygon, 5)
            dictionary['soil_area'].append(soil_area)

            total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
            dictionary['total_area'].append(total_area)
            
            if ndvi:
                tree_ndvi_mean, tree_ndvi_max, tree_ndvi_min = ndvi_calculation(imagery, 1, polygon, ref_image = im).getInfo().values()
                dictionary['tree_ndvi_mean'].append(tree_ndvi_mean)
                dictionary['tree_ndvi_max'].append(tree_ndvi_max)
                dictionary['tree_ndvi_min'].append(tree_ndvi_min)

                grass_ndvi_mean, grass_ndvi_max, grass_ndvi_min = ndvi_calculation(imagery, 2, polygon, ref_image = im).getInfo().values()
                dictionary['grass_ndvi_mean'].append(grass_ndvi_mean)
                dictionary['grass_ndvi_max'].append(grass_ndvi_max)
                dictionary['grass_ndvi_min'].append(grass_ndvi_min)



            end = time()
            print(f'Zip Code: {i}, Year: {j[0][:4]} ::: completed in {end-start} seconds.')
            
    return dictionary
              
              

In [31]:
inference_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2018,2020],
        'zipcodes': ['90802','90732'],
        'ndvi': False,
        'residential': False,
        'residential_shape': la_parcel_res, #don't adjust this line
        'counties': {'la_county': la_county}, #don't adjust this line
        'zipcode_shape' : la_county_income_zipcode #don't adjust
         }

In [ ]:
dictionary = run_inference(inference_params)

#### Inference - City Level

In [32]:
#load California Cities Shapefile
cities = ee.FeatureCollection("projects/california-lawn-detection/assets/Cities2015")

# Filter for LA City
la_cities = cities.filter(ee.Filter.eq('County', 'Los Angeles'))

# Filter sea
la_cities_land = la_cities.filter(ee.Filter.notEquals('Notes', '3 nautical mile offshore'))

# list of cities in LA County
city_names = []

for i in range(len(la_cities_land.getInfo().get('features'))):
    city_names.append(la_cities_land.getInfo().get('features')[i].get('properties').get('NAME'))

In [33]:
# for cities example - we can replace city : 'Pasadena' with city_names
inference_params_city = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2010,2020],
        'city': city_names,
        'ndvi': False,
        'residential': False,
        'residential_shape': la_parcel_res, #don't adjust this line
        'counties': {'la_county': la_county}, #don't adjust this line
        'city_shape' : la_cities_land #don't adjust
         }

In [34]:
# for cities
dictionary = run_inference_city(inference_params_city)

City: Los Angeles, Year: 2010 ::: completed in 487.4633662700653 seconds.
City: Los Angeles, Year: 2020 ::: completed in 467.0242302417755 seconds.
City: Pasadena, Year: 2010 ::: completed in 183.75258588790894 seconds.
City: Pasadena, Year: 2020 ::: completed in 2.8775172233581543 seconds.
City: Santa Monica, Year: 2010 ::: completed in 43.30096650123596 seconds.
City: Santa Monica, Year: 2020 ::: completed in 43.44421362876892 seconds.
City: Monrovia, Year: 2010 ::: completed in 78.5770206451416 seconds.
City: Monrovia, Year: 2020 ::: completed in 91.05835509300232 seconds.
City: Pomona, Year: 2010 ::: completed in 128.29264545440674 seconds.
City: Pomona, Year: 2020 ::: completed in 125.50807094573975 seconds.
City: South Pasadena, Year: 2010 ::: completed in 23.034969329833984 seconds.
City: South Pasadena, Year: 2020 ::: completed in 24.432021856307983 seconds.
City: Compton, Year: 2010 ::: completed in 59.677114725112915 seconds.
City: Compton, Year: 2020 ::: completed in 61.7532

City: Bradbury, Year: 2020 ::: completed in 18.26105809211731 seconds.
City: Irwindale, Year: 2010 ::: completed in 64.89083313941956 seconds.
City: Irwindale, Year: 2020 ::: completed in 67.24306297302246 seconds.
City: Duarte, Year: 2010 ::: completed in 49.77750587463379 seconds.
City: Duarte, Year: 2020 ::: completed in 61.36676645278931 seconds.
City: Norwalk, Year: 2010 ::: completed in 51.496668100357056 seconds.
City: Norwalk, Year: 2020 ::: completed in 61.3387885093689 seconds.
City: Bellflower, Year: 2010 ::: completed in 33.025707721710205 seconds.
City: Bellflower, Year: 2020 ::: completed in 37.15795373916626 seconds.
City: Rolling Hills Estates, Year: 2010 ::: completed in 38.52923631668091 seconds.
City: Rolling Hills Estates, Year: 2020 ::: completed in 41.998462438583374 seconds.
City: Pico Rivera, Year: 2010 ::: completed in 67.53255319595337 seconds.
City: Pico Rivera, Year: 2020 ::: completed in 71.62167572975159 seconds.
City: South El Monte, Year: 2010 ::: comple

In [35]:
df = pd.DataFrame(dictionary)
df

,year,polygon,water_area,vegetation_trees_area,vegetation_grass_area,turf_area,impervious_area,soil_area,total_area
0,2010,Los Angeles,0.324202,130.385387,353.605794,0.060126,410.697858,333.586243,1228.659609
1,2020,Los Angeles,1.626241,309.909160,95.781669,8.186225,619.210499,193.945815,1228.659609
2,2010,Pasadena,0.004953,6.878802,18.366940,0.000330,18.924296,15.665165,59.840485
3,2020,Pasadena,0.004953,26.888325,6.511651,0.155270,15.800284,10.480002,59.840485
4,2010,Santa Monica,0.000992,1.745463,3.223802,0.009591,11.455411,5.696855,22.132114
...,...,...,...,...,...,...,...,...,...
179,2020,Diamond Bar,0.000331,12.466278,6.526314,0.092141,12.895578,6.541375,38.522016
180,2010,Malibu,0.000442,2.118154,22.759438,0.000165,18.022123,8.235701,51.136024
181,2020,Malibu,0.361275,5.808764,13.845387,0.048684,16.613993,14.457920,51.136024
182,2010,Calabasas,0.006604,3.080692,13.142739,0.000660,9.845636,9.687033,35.763364
